In [111]:
import pandas as pd
import os
import numpy as np  # si tu n'en as pas besoin ailleurs, tu peux enlever cet import

def load_spectrum_table_only(path: str) -> pd.DataFrame | None:
    """
    Charge un fichier .txt de spectroscopie Raman et retourne
    le tableau brut (après nettoyage léger) sous forme de DataFrame.

    - Repère la ligne 'Pixel;' pour trouver le début des données.
    - Lit le tableau avec les mêmes paramètres que load_spectrum_file.
    - Supprime la dernière colonne si elle est 'Unnamed'.
    - Nettoie les noms de colonnes et tente la conversion numérique.

    Args:
        path: chemin vers le fichier .txt

    Returns:
        DataFrame contenant le tableau lu
        ou None si erreur / format non reconnu.
    """
    try:
        # --- Lecture brute + repérage de la ligne "Pixel;" ---
        with open(path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # On repère la première ligne qui commence par "Pixel;"
        header_idx = next(
            i for i, line in enumerate(lines)
            if line.strip().startswith("Pixel;")
        )

        # Lecture du tableau à partir de cette ligne
        df = pd.read_csv(
            path,
            skiprows=header_idx,
            sep=";",           # comme dans load_spectrum_file
            decimal=",",
            encoding="utf-8",
            skipinitialspace=True,
            na_values=["", " ", "   ", "\t"],
            keep_default_na=True,
        )

        # Supprimer dernière colonne si vide / Unnamed
        if df.columns[-1].startswith("Unnamed"):
            df = df.iloc[:, :-1]

        # Nettoyage des noms de colonnes (espaces, insécables, etc.)
        df.columns = [str(c).strip().replace("\xa0", " ") for c in df.columns]

        # Conversion numérique des colonnes texte (comme dans ta fonction)
        for col in df.columns:
            if df[col].dtype == "object":
                df[col] = pd.to_numeric(df[col], errors="coerce")

        # On ne fait rien de plus : on retourne juste le tableau
        return df

    except StopIteration:
        # Si on ne trouve pas "Pixel;", on le signale et on renvoie None
        print(f"[load_spectrum_table_only] Ligne 'Pixel;' introuvable dans {path}")
        return None

    except Exception as e:
        print(f"Erreur lors du chargement de {path}: {e}")
        return None

In [112]:
spectre_03 = load_spectrum_table_only("/Users/souchaud/Documents/Travail/CitizenSers/Spectroscopie/2025_12_02_Formation/GC523/GC523/GC523_06.txt")

In [113]:
spectre_03

,Pixel,Wavelength,Wavenumber,Raman Shift,Dark,Reference,Raw data #1,Dark Subtracted #1,%TR #1,Absorbance #1,Irradiance (lumen) #1
0,0,NaN,NaN,NaN,1144.6667,65535.0,1179.0000,34.3333,0.0,0.0,0.0
1,1,NaN,NaN,NaN,1215.0000,65535.0,1235.0000,20.0000,0.0,0.0,0.0
2,2,NaN,NaN,NaN,1133.6667,65535.0,1187.6667,54.0000,0.0,0.0,0.0
3,3,NaN,NaN,NaN,1141.0000,65535.0,1179.6667,38.6667,0.0,0.0,0.0
4,4,NaN,NaN,NaN,1211.3333,65535.0,1265.6667,54.3333,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2043,2043,NaN,NaN,NaN,965.3333,65535.0,997.0000,31.6667,0.0,0.0,0.0
2044,2044,NaN,NaN,NaN,872.3333,65535.0,937.0000,64.6667,0.0,0.0,0.0
2045,2045,NaN,NaN,NaN,988.0000,65535.0,969.6667,-18.3333,0.0,0.0,0.0
2046,2046,NaN,NaN,NaN,966.6667,65535.0,979.6667,13.0000,0.0,0.0,0.0


In [114]:
spectre_ok = load_spectrum_table_only("/Users/souchaud/Documents/Travail/CitizenSers/Spectroscopie/2025_12_02_Formation/GC524/GC524/GC524_03.txt")

In [115]:
# 1) Récupération des colonnes de remplacement
cols_ok = spectre_ok.iloc[:, :4]   # 3 premières colonnes

# 2) Suppression des 4 premières colonnes de spectre_03
spectre_03_rest = spectre_03.iloc[:, 4:]   # colonnes restantes après les 4 premières

# 3) Reconstruction du DataFrame
spectre_03_new = pd.concat([cols_ok, spectre_03_rest], axis=1)

In [116]:
def load_spectrum_header(path: str) -> str | None:
    """
    Retourne uniquement la partie texte avant la ligne commençant par 'Pixel;'.
    Aucun traitement supplémentaire.

    Args:
        path : chemin du fichier .txt

    Returns:
        Le texte avant 'Pixel;' sous forme de chaîne,
        ou None si la structure n'est pas trouvée.
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Trouver où commence la zone de données
        header_idx = next(
            i for i, line in enumerate(lines)
            if line.strip().startswith("Pixel;")
        )

        # Retourner tout ce qu'il y a avant
        header_text = "".join(lines[:header_idx])
        return header_text

    except StopIteration:
        print(f"[load_spectrum_header] Ligne 'Pixel;' introuvable dans {path}")
        return None
    except Exception as e:
        print(f"Erreur lors de la lecture de {path}: {e}")
        return None

In [117]:
spectre_header = load_spectrum_header("/Users/souchaud/Documents/Travail/CitizenSers/Spectroscopie/2025_12_02_Formation/GC523/GC523/GC523_03.txt")

In [118]:
spectre_header

'File Version;BWSpec4.02\nDate;1980-09-05 03:45:25\ntitle;BWS415-532S\nmodel;BTC162E-532S-SYS\nc code;RLZ\noperator;\nport1;0\nbaud1;3\npixel_start;0\npixel_end;2047\nstep;1\nunits;3\nbkcolor;16777215\nshow_mode;3\ndata_mode;0\npixel_mode;0\nintigration times(ms);7000\naverage number;3\ntime_multiply;1\nspectrometer_type ;14\nyaxis;1\nyaxis_min;0\nyaxis_max;12000\nxaxis;1\nxaxis_min;460\nxaxis_max;502\nirrands_DispWLMin;100\nirrands_DispWLMax;1000\nyaxis_min_6;0\nyaxis_max_6;0\nirradiance_unit;0\nColor_Data_Flag;0\nColor_StartWL;532\nColor_EndWL;684\nColor_IncWL;10\npower_unit_index;3\nphotometric_index;0\nIlluminant_index;3\nobserver_index;0\nlab_l;0\nlab_a;0\nlab_b;0\nradiometric_flag;0\ncoefs_a0;531,016606800964\ncoefs_a1;0,0821624253029554\ncoefs_a2;-1,81293471677812E-06\ncoefs_a3;-8,76279014177826E-10\ncoefs_b0;-12989,5740197986\ncoefs_b1;48,9548877772458\ncoefs_b2;-0,0693497593775465\ncoefs_b3;4,37280052698368E-05\nall_data_save;1\nselect_option;-1\ninterval_time;7000\npixel_num;

In [119]:
def save_spectrum_with_header(path_out: str, header: str, df: pd.DataFrame) -> None:
    """
    Sauvegarde un fichier texte avec :
    - le header (texte brut)
    - puis le DataFrame en CSV ; avec décimale , et \n comme saut de ligne.
    """
    # 1) Écrire le header
    with open(path_out, "w", encoding="utf-8") as f:
        # on s'assure qu'il se termine bien par un saut de ligne
        if not header.endswith("\n"):
            header = header + "\n"
        f.write(header)

    # 2) Ajouter le tableau après
    df.to_csv(
        path_out,
        mode="a",              # append après le header
        sep=";",               # séparateur ;
        index=False,           # pas de colonne d'index pandas
        lineterminator="\n",  # sauts de ligne
        decimal=",",           # virgule comme séparateur décimal
        encoding="utf-8"       # même encodage
    )

In [120]:
save_spectrum_with_header("./mon_fichier.txt", spectre_header, spectre_03_new)

In [121]:
spectre_03_new.to_csv("./mon_fichier.txt", sep=";", index=False, lineterminator="\n", encoding="utf-8")

In [122]:
spectre_03

,Pixel,Wavelength,Wavenumber,Raman Shift,Dark,Reference,Raw data #1,Dark Subtracted #1,%TR #1,Absorbance #1,Irradiance (lumen) #1
0,0,NaN,NaN,NaN,1144.6667,65535.0,1179.0000,34.3333,0.0,0.0,0.0
1,1,NaN,NaN,NaN,1215.0000,65535.0,1235.0000,20.0000,0.0,0.0,0.0
2,2,NaN,NaN,NaN,1133.6667,65535.0,1187.6667,54.0000,0.0,0.0,0.0
3,3,NaN,NaN,NaN,1141.0000,65535.0,1179.6667,38.6667,0.0,0.0,0.0
4,4,NaN,NaN,NaN,1211.3333,65535.0,1265.6667,54.3333,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2043,2043,NaN,NaN,NaN,965.3333,65535.0,997.0000,31.6667,0.0,0.0,0.0
2044,2044,NaN,NaN,NaN,872.3333,65535.0,937.0000,64.6667,0.0,0.0,0.0
2045,2045,NaN,NaN,NaN,988.0000,65535.0,969.6667,-18.3333,0.0,0.0,0.0
2046,2046,NaN,NaN,NaN,966.6667,65535.0,979.6667,13.0000,0.0,0.0,0.0
